# Unisens Data Test

This notebok is used to explore the data the unisens data fromat.

In [ ]:
import unisens

# folder containing the unisens.xml file
data_path = '../data/unisens-test/'
data = unisens.Unisens(data_path, readonly=True)

In [ ]:
# SPO2      --> oxygen saturation
# Pleth     --> plethysmogram
# Pulse     --> pulse rate
# EKG       --> electrocardiogram
# Lux Meter --> light intensity

In [ ]:
print(data.entries)

In [ ]:
print(data['8_Thorax_Abdomen'].attrib)

In [ ]:
# The sample rate of the data in Hz
sample_rate = int(data['8_Thorax_Abdomen'].attrib['sampleRate'])

In [ ]:
# Get information about the data
for key in data['8_Thorax_Abdomen.bin']:
    print(key)

In [ ]:
# Get the data from the unisens file
thorax_abdomen_data = data['8_Thorax_Abdomen'].get_data()
print(type(thorax_abdomen_data))
print(thorax_abdomen_data.shape)

In [ ]:
import plotly.express as px

# Flatten the data to a 1D array
thorax_abdomen_flat = thorax_abdomen_data.flatten()

# Normalize the signal between -1 and 1
thorax_abdomen_flat = ((thorax_abdomen_flat - thorax_abdomen_flat.min()) /
                       (thorax_abdomen_flat.max() - thorax_abdomen_flat.min()) * 2 - 1)

# Plot the thorax abdomen data
fig = px.line(x=range(len(thorax_abdomen_flat)), y=thorax_abdomen_flat)
fig.show()

In [ ]:
from scipy.signal import find_peaks

# Get the peaks in the signal
peaks, _ = find_peaks(thorax_abdomen_flat, height=0.5)

# Calculate the breathing rate per minute
beat_rate = len(peaks) / (len(thorax_abdomen_flat) / sample_rate / 60)
beat_rate

In [ ]:
# Plot the data with the peaks
fig = px.line(x=range(len(thorax_abdomen_flat)), y=thorax_abdomen_flat)
fig.add_scatter(x=peaks, y=thorax_abdomen_flat[peaks], mode='markers')
fig.show()

In [ ]:
# Calculate the fast fourier transform
import numpy as np
from scipy.fft import fft

# Calculate the fast fourier transform of the thorax abdomen data
thorax_abdomen_fft = fft(thorax_abdomen_flat)

# Calculate the frequency
freq = np.fft.fftfreq(len(thorax_abdomen_fft), 1 / sample_rate)

# Get the positive frequencies
mask = freq > 0
freq = freq[mask]

# Get the maximum frequency
max_freq = freq[np.argmax(np.abs(thorax_abdomen_fft[mask]))]

# Calculate the breathing rate from the maximum frequency per minute
breathing_rate = max_freq * 60
breathing_rate